# Running TF Profiler (w/ Keras)

### Prerequisites

- Python 3.6
- Keras <= 2.1.6
- Compatible Tensorflow
- Profiler GUI prerequisites: https://github.com/tensorflow/profiler-ui. 

It's important that you install the right prerequisites Keras doesn't currently support runtime options: https://github.com/tensorflow/tensorflow/issues/19911

The rest of the necessary libraries (and Python) come preinstalled with Anaconda and it's recommended that you setup the prerequisites in a virtual environment so it doesn't interfere with other projects.

In [1]:
import keras
import tensorflow as tf

C:\Users\vsunkavalli\AppData\Local\conda\conda\envs\profilerenv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Creating your model

Make sure that you compile your models with tensorflow runtime options like shown:

    run_metadata = tf.RunMetadata()
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    model.compile(..., run_metadata=run_metadata, run_options=run_options)


In [2]:
#Loading Data - Unimportant
from keras.datasets import cifar10
num_classes=10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train[0:512]
y_train = y_train[0:512]
x_test = x_test[0:64]
y_test = y_test[0:64]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (512, 32, 32, 3)
512 train samples
64 test samples


In [3]:
#Loading Model - Unimportant
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

batch_size = 32
epochs = 3


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [8]:
run_metadata = tf.RunMetadata()
run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],
              run_metadata=run_metadata, options=run_options)

### Profiling your model

When training your model, you must setup the code in a particular way in order to get the profile log. Make sure to follow the code provided and only edit the train statement.

In [9]:
log_dir = './' #Set up the directory where you want to store your profiles

In [10]:
#Using TF Profiler
builder = tf.profiler.ProfileOptionBuilder
opts = builder(builder.time_and_memory()).order_by('micros').build()
opts2 = tf.profiler.ProfileOptionBuilder.trainable_variables_parameter()


with tf.contrib.tfprof.ProfileContext(
    log_dir,
    trace_steps=range(10,20),
    dump_steps=[20]) as pctx:

    pctx.add_auto_profiling('op', opts, [15,18,20])
    pctx.add_auto_profiling('scope', opts2, [20])
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test),shuffle=True)

Train on 512 samples, validate on 64 samples
Epoch 1/3
256/512 [==============>...............] - ETA: 2s - loss: 2.3187 - acc: 0.1016

287 ops no flops stats due to incomplete shapes.


512/512 [==============================] - 74s 144ms/step - loss: 2.3109 - acc: 0.0996 - val_loss: 2.2931 - val_acc: 0.1250
Epoch 2/3
512/512 [==============================] - 11s 22ms/step - loss: 2.2936 - acc: 0.1211 - val_loss: 2.2905 - val_acc: 0.0469
Epoch 3/3
512/512 [==============================] - 5s 9ms/step - loss: 2.2872 - acc: 0.1250 - val_loss: 2.2806 - val_acc: 0.0625


### Running the Tensorflow Profiler/Timeline

Make sure you have Chrome installed in order to use the profiler. In order to access the profiler GUI, you will need to use the Anaconda Prompt and navigate to the profiler-ui folder, then run ui.py and give it a path to your profile. Details are listed here: https://github.com/tensorflow/profiler-ui. 